# Comparar archivos de etiquetado  

Comparador de etiquetas generadas por convert_to_yolo_d.v3 con formato YOLO Darknet (archivo JSON), y el etiquetado original 

> Input
*   Directorio que contiene los archivos de etiquetado en formato YOLO Darknet (archivos .txt)
*   Ancho de las imagenes del dataset contenidas en el directorio padre
*   Alto de las imagenes del dataset contenidas en el directorio padre

> Output
*   Etiquetado en formato original (berries_v8.json)
*   Comparativa en % entre el etiquetado original y el que fue generado en esta aplicacion


In [ ]:
#parametros iniciales necesarios

directorio = '/content/drive/MyDrive/PT/dataset/IMG_1123/labels'    #directorio que contiene los archivos de etiquetado en formato YOLO Darknet (archivos .txt)
w_i = 1920   #ancho de imagenes en entero, ej: 3264
h_i = 1080    #alto de imagenes en entero, ej: 2448

archivo_json_v2 = 'berries_v2.json'    #nuevo archivo json que contendra las etiquetas de los bounding box en formato original

In [ ]:
#importacion de bibliotecas necesarias

import json
import os
from PIL import Image
from PIL.ExifTags import TAGS  
from pathlib import Path

In [ ]:
#definicion de variables 

w = w_i
h = h_i    #se asignan valores recibidos de ancho y alto a variables auxiliares 

directorio = directorio.replace('\\', '/') + "/"    #se modifica el directorio para correcta lectura de python
contenido_directorio = os.listdir(directorio)    #se almacena en una lista el nombre de todos los archivos contenidos en el directorio
print("Directorio de etiquetas: \n", directorio)   
print("Contenido del directorio: \n", contenido_directorio)   

directorio_p = str(Path(directorio).parents[0])+ "/"    #se accede al directorio padre el que contiene el archivo JSON con etiquetas original
contenido_directorio_p = os.listdir(directorio_p)    #se almacena en una lista el nombre de todos los archivos contenidos en el directorio padre
print("Directorio de dataset con archivo JSON original: \n", directorio_p)   
print("Contenido del directorio: \n", contenido_directorio_p)   

data = {}
data_v2 = {}
nombre_archivos_jpg = []    
nombre_archivos_json = []
elementos_linea = []    
etxt = '.txt'
ejpg = '.jpg'  
ejson = '.json'
count = 0     #contador de bounding boxes
similitud = 0    #similitud promedio de etiquetas originales vs las generadas en este script

Directorio de etiquetas: 
 /content/drive/MyDrive/PT/dataset/IMG_1123/labels/
Contenido del directorio: 
 ['frame0764.txt', 'frame0710.txt', 'frame0806.txt', 'frame0665.txt', 'frame0653.txt', 'frame0658.txt', 'frame0642.txt', 'frame0616.txt', 'frame0607.txt', 'frame0582.txt', 'frame0630.txt', 'frame0479.txt', 'frame0516.txt', 'frame0358.txt', 'frame0298.txt', 'frame0281.txt', 'frame0239.txt', 'frame0269.txt', 'frame0544.txt', 'frame0469.txt', 'frame0404.txt', 'frame0393.txt', 'frame0386.txt', 'frame0458.txt', 'frame0440.txt', 'frame0127.txt', 'frame0111.txt', 'frame0191.txt', 'frame0135.txt', 'frame0219.txt', 'frame0206.txt', 'frame0371.txt', 'frame0232.txt', 'frame0000.txt', 'frame1462.txt', 'frame1495.txt', 'frame1450.txt', 'frame1435.txt', 'frame1422.txt', 'frame1398.txt', 'frame1429.txt', 'frame1341.txt', 'frame1320.txt', 'frame1347.txt', 'frame1269.txt', 'frame1286.txt', 'frame1115.txt', 'frame1160.txt', 'frame1187.txt', 'frame1275.txt', 'frame1230.txt', 'frame1076.txt', 'frame105

In [ ]:
#comprobacion de archivos en directorio

for archivo in contenido_directorio:
    if os.path.isfile(os.path.join(directorio, archivo)) and archivo.endswith(etxt):
        nombre_archivos_jpg.append(archivo.replace(etxt, ''))    #se obtienen los nombres de los archivos y se almacenan sin la extension '.jpg'
        print("Archivo TXT encontrado: ", archivo)    #se comprueba el nombre de los archivos seleccionados

for archivo in contenido_directorio_p:
    if os.path.isfile(os.path.join(directorio_p, archivo)) and archivo.endswith(ejson):
        archivo_json = archivo    #se obtiene solo el primer archivo .json encontrado (etiquetado original)
        print("Archivo JSON encontrado: ", archivo)    #se comprueba el nombre de los archivos seleccionados
        break    

etiquetas = directorio_p + archivo_json
etiquetas_v2 = directorio_p + archivo_json_v2

Archivo TXT encontrado:  frame0764.txt
Archivo TXT encontrado:  frame0710.txt
Archivo TXT encontrado:  frame0806.txt
Archivo TXT encontrado:  frame0665.txt
Archivo TXT encontrado:  frame0653.txt
Archivo TXT encontrado:  frame0658.txt
Archivo TXT encontrado:  frame0642.txt
Archivo TXT encontrado:  frame0616.txt
Archivo TXT encontrado:  frame0607.txt
Archivo TXT encontrado:  frame0582.txt
Archivo TXT encontrado:  frame0630.txt
Archivo TXT encontrado:  frame0479.txt
Archivo TXT encontrado:  frame0516.txt
Archivo TXT encontrado:  frame0358.txt
Archivo TXT encontrado:  frame0298.txt
Archivo TXT encontrado:  frame0281.txt
Archivo TXT encontrado:  frame0239.txt
Archivo TXT encontrado:  frame0269.txt
Archivo TXT encontrado:  frame0544.txt
Archivo TXT encontrado:  frame0469.txt
Archivo TXT encontrado:  frame0404.txt
Archivo TXT encontrado:  frame0393.txt
Archivo TXT encontrado:  frame0386.txt
Archivo TXT encontrado:  frame0458.txt
Archivo TXT encontrado:  frame0440.txt
Archivo TXT encontrado:  

In [ ]:
#analisis y conversion de puntos contenidos en archivos TXT al formato original

print("Archivos analizados: ")
for nombre in nombre_archivos_jpg:    #para cada nombre almacenado en 'nombre_archivos_jpg', obtenidos a partir del dataset de imagenes
    archivo_txt = directorio + nombre + etxt  
    imagen = nombre + ejpg  
    data_v2[imagen] = []
    archivo_jpg = directorio_p + imagen    #ubicacion de cada imagen para lectura de metadatos

    image = Image.open(archivo_jpg)    #Lee la imagen en el directorio señalado
    exifdata = image.getexif() 
    for tagid in exifdata: 
        tagname = TAGS.get(tagid, tagid) 
        value = exifdata.get(tagid) 
        if (tagname == "Orientation" and value == 1):    #si la orientacion de la imagen es horizontal
            w = w_i
            h = h_i
        if (tagname == "Orientation" and value == 6):    #si la orientacion de la imagen es vertical
            w = h_i
            h = w_i
        #print(f"{tagname:25}: {value}") 
    image.close()

    with open(archivo_txt) as txt:
        for linea in txt:
            elementos_linea = linea.split()
            #print("elementos_linea: ", elementos_linea)

            bbox_x = float(elementos_linea[1])
            bbox_y = float(elementos_linea[2])
            bbox_w = float(elementos_linea[3])
            bbox_h = float(elementos_linea[4])

            punto_x_v2 = w*(bbox_x - (bbox_w/2))
            punto_y_v2 = h*(bbox_y - (bbox_h/2))
            punto_w_v2 = punto_x_v2 + w*bbox_w
            punto_h_v2 = punto_y_v2 + h*bbox_h
            #print("nueva linea: ", punto_x_v2, punto_y_v2, punto_w_v2, punto_h_v2) 

            data_v2[imagen].append({
                'x': round(punto_x_v2),
                'y': round(punto_y_v2),
                'w': round(punto_w_v2),
                'h': round(punto_h_v2)})
        print(" ", archivo_txt)

Archivos analizados: 
  /content/drive/MyDrive/PT/dataset/IMG_1123/labels/frame0764.txt
  /content/drive/MyDrive/PT/dataset/IMG_1123/labels/frame0710.txt
  /content/drive/MyDrive/PT/dataset/IMG_1123/labels/frame0806.txt
  /content/drive/MyDrive/PT/dataset/IMG_1123/labels/frame0665.txt
  /content/drive/MyDrive/PT/dataset/IMG_1123/labels/frame0653.txt
  /content/drive/MyDrive/PT/dataset/IMG_1123/labels/frame0658.txt
  /content/drive/MyDrive/PT/dataset/IMG_1123/labels/frame0642.txt
  /content/drive/MyDrive/PT/dataset/IMG_1123/labels/frame0616.txt
  /content/drive/MyDrive/PT/dataset/IMG_1123/labels/frame0607.txt
  /content/drive/MyDrive/PT/dataset/IMG_1123/labels/frame0582.txt
  /content/drive/MyDrive/PT/dataset/IMG_1123/labels/frame0630.txt
  /content/drive/MyDrive/PT/dataset/IMG_1123/labels/frame0479.txt
  /content/drive/MyDrive/PT/dataset/IMG_1123/labels/frame0516.txt
  /content/drive/MyDrive/PT/dataset/IMG_1123/labels/frame0358.txt
  /content/drive/MyDrive/PT/dataset/IMG_1123/labels/fr

In [ ]:
#almacenamiento en archivo JSON de etiquetas re convertidas

with open(etiquetas_v2, 'w') as file:
    json.dump(data_v2, file, indent=4)
    print("Se han convertido los archivos .txt al formato de etiquetado original y se han almacenado en: ")
    print(etiquetas_v2)

Se han convertido los archivos .txt al formato de etiquetado original y se han almacenado en: 
/content/drive/MyDrive/PT/dataset/IMG_1123/berries_v2.json


In [ ]:
#comparacion entre las etiquetas originales y las etiquetas re convertidas, contenidas en sus archivos JSON respectivos

with open(etiquetas) as file, open(etiquetas_v2) as file_2:
    data = json.load(file)
    data_v2 = json.load(file_2)

    for nombre in nombre_archivos_jpg:    #para cada nombre almacenado en 'nombre_archivos_jpg', obtenidos a partir del dataset de imagenes
        imagen = nombre + ejpg    #imagen es el "objeto" a buscar en el json (nombre de la imagen, cuyo etiquetado esta contenido en el objeto)

        for objeto in data[imagen]:
            punto_x = int(objeto['x'])    #los puntos obtenidos del archivo json se transforman a formato numerico
            punto_y = int(objeto['y'])
            punto_w = int(objeto['w'])
            punto_h = int(objeto['h'])

        for objeto_2 in data_v2[imagen]:
            punto_x_2 = int(objeto_2['x'])    #los puntos obtenidos del archivo json se transforman a formato numerico
            punto_y_2 = int(objeto_2['y'])
            punto_w_2 = int(objeto_2['w'])
            punto_h_2 = int(objeto_2['h'])

        if punto_x != punto_x_2:
            similitud = similitud + (100*(punto_x_2/punto_x))
        else:
            similitud = similitud + 100

        if punto_y != punto_y_2:
            similitud = similitud + (100*(punto_y_2/punto_y))
        else:
            similitud = similitud + 100

        if punto_w != punto_w_2:
            similitud = similitud + (100*(punto_w_2/punto_w))
        else:
            similitud = similitud + 100

        if punto_h != punto_h_2:
            similitud = similitud + (100*(punto_h_2/punto_h))
        else:
            similitud = similitud + 100
        count = count + 4
    
    similitud = round((similitud / count), 3)
    print("Comparando '", etiquetas, "' con '", etiquetas_v2, "'")
    print("\nLas etiquetas convertidas son ", similitud, "%", " similares a las del archivo de etiquetas .json original.")

Comparando ' /content/drive/MyDrive/PT/dataset/IMG_1123/berries.json ' con ' /content/drive/MyDrive/PT/dataset/IMG_1123/berries_v2.json '

Las etiquetas convertidas son  100.0 %  similares a las del archivo de etiquetas .json original.
